In [1]:
#-----IMPORTANDO ARQUIVO CSV E CRIANDO DATA FRAME 'df_ocorrencia_tipo'-----#

import os 
import pandas as pd
import numpy as np 
import psycopg2 as pg

df_ocorrencia_tipo = pd.read_csv('C:\\Users\\josen\\Desktop\\Projetos\\ocorrencia_tipo.csv', delimiter=';', encoding='utf-8')

In [2]:
#-----CORREÇÃO DE ERROS NA FORMATAÇÃO DOS DADOS-----#

#-----Células com dados inválidos
df_ocorrencia_tipo = df_ocorrencia_tipo.replace('***', '')

In [6]:
#-----CONECTANDO AO BANCO DE DADOS-----#

host = 'localhost'
dbname = 'projeto_acidentes_aeroviarios'
user = '********'
password = '*******'
sslmode = 'disable'

conect_string = 'host={} user={} dbname={} password={} sslmode={}'.format(host, user, dbname, password, sslmode)
conec = pg.connect(conect_string)
cursor = conec.cursor()

print('Conectado ao banco de dados {} com sucesso'.format(dbname))

Conectado ao banco de dados projeto_acidentes_aeroviarios com sucesso


In [4]:
#-----GERANDO NOMES DAS COLUNAS PARA AUTOMATIZAR O COMANDO DE CRIAÇÃO DE TABELA-----#

tipos_de_dados_formatpostgr = {
    'object' : 'varchar',
    'int32' : 'integer',
    'int64' : 'integer',
    'datetime64[ns]' : 'date',
    'float64' : 'numeric'
}
colunas_formatadas = ', '.join('{} {}'.format(n, d) for (n, d) in zip(df_ocorrencia_tipo.columns, df_ocorrencia_tipo.dtypes.replace(tipos_de_dados_formatpostgr)))
colunas_ocorrencia_tipo = colunas_formatadas.replace('horario varchar', 'horario time with time zone')

In [5]:
#-----CRIANDO NOVA TABELA DENTRO DO BANCO DE DADOS-----#

nome_da_tabela = 'ocorrencia_tipo'
colunas = colunas_ocorrencia_tipo
sql_querry = 'create table {} ({})'.format(nome_da_tabela, colunas)
cursor.execute(sql_querry)
conec.commit()
cursor.close()
conec.close()

In [7]:
#-----SALVANDO O DATA FRAME 'df_ocorrencia_tipo' DENTRO DA TABELA 'ocorrencia_tipo'-----#

df_ocorrencia_tipo.to_csv('ocorrencia_tipo', header=df_ocorrencia_tipo.columns, index=False, encoding='utf-8')
arquivo_aberto_para_upload = open('ocorrencia_tipo', encoding='utf-8')

consulta_sql_para_upload = """
COPY ocorrencia_tipo FROM STDIN WITH
    CSV HEADER
    DELIMITER ','
    ENCODING 'utf-8'
"""
cursor.copy_expert(sql=consulta_sql_para_upload, file=arquivo_aberto_para_upload)
conec.commit()
cursor.close()
conec.close()
print('Arquivo adicionado a tabela {} no banco de dados {} com sucesso'.format(nome_da_tabela, dbname))

Arquivo adicionado a tabela ocorrencia_tipo no banco de dados projeto_acidentes_aeroviarios com sucesso
